# Hybrid Framework Approaches: Combining AI Agent Systems

This notebook demonstrates how to build hybrid AI agent systems that combine multiple frameworks. These examples show how to leverage the strengths of different frameworks while overcoming their individual limitations.

## Helpful Resources

- [LangChain AI Agent Development Guide](https://python.langchain.com/docs/integrations/agents/)
- [Microsoft Research: Foundation of Autonomous Agents](https://www.microsoft.com/en-us/research/group/autonomous-systems-group-robotics/articles/foundation-of-autonomous-agents/)
- [Autonomous Agent Architectures Blog](https://lilianweng.github.io/posts/2023-06-23-agent/)
- [Building LLM-powered Autonomous Agents](https://blog.langchain.dev/building-llm-powered-autonomous-agents/)
- [Stanford Agent Framework Overview](https://crfm.stanford.edu/2023/06/16/agent-framework.html)
- [LlamaIndex Agent Framework](https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/)

## Setup Instructions

First, let's install all the required packages for the different frameworks we'll be using:

In [ ]:
# Install required packages
!pip install langchain langgraph crewai pyautogen langchain_openai
!pip install wikipedia transformers sentence-transformers

## Setting API Keys and Configurations

For these examples to work, you'll need to set your API keys for OpenAI or Azure OpenAI.

In [ ]:
import os

# Option 1: OpenAI Setup
# Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"  # Replace with your actual API key

# Option 2: Azure OpenAI Setup
# Set your Azure OpenAI variables
# os.environ["AZURE_OPENAI_API_KEY"] = "your-azure-openai-api-key"  # Replace with your Azure API key
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource-name.openai.azure.com/"  # Replace with your endpoint

# Helper function to create appropriate clients based on provider choice
def create_clients(provider="openai"):
    """Create OpenAI or Azure OpenAI clients for different frameworks."""
    from langchain_openai import ChatOpenAI, AzureChatOpenAI
    
    if provider == "azure":
        # Azure OpenAI setup
        langchain_llm = AzureChatOpenAI(
            azure_deployment="gpt-4",  # The deployment name you chose
            openai_api_version="2023-05-15",
            azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", ""),
            api_key=os.environ.get("AZURE_OPENAI_API_KEY", ""),
            temperature=0.7
        )
        
        # Azure OpenAI config for AutoGen
        autogen_config = {
            "config_list": [{
                'model': 'gpt-4',  # Your deployment name
                'api_type': 'azure',
                'api_version': '2023-05-15',
                'api_key': os.environ.get("AZURE_OPENAI_API_KEY", ""),
                'api_base': os.environ.get("AZURE_OPENAI_ENDPOINT", "")
            }]
        }
    else:  # Default to OpenAI
        # Standard OpenAI setup
        langchain_llm = ChatOpenAI(
            model="gpt-4",
            temperature=0.7,
            api_key=os.environ.get("OPENAI_API_KEY", "")
        )
        
        # OpenAI config for AutoGen
        autogen_config = {
            "config_list": [{
                'model': 'gpt-4',
                'api_key': os.environ.get("OPENAI_API_KEY", "")
            }]
        }
    
    return {
        "langchain_llm": langchain_llm,
        "autogen_config": autogen_config
    }

# Choose your provider
provider = "openai"  # Change to "azure" to use Azure OpenAI

# Get the appropriate clients
clients = create_clients(provider)

## Pattern 1: LangChain + LangGraph

This example demonstrates how to combine LangChain's tools with LangGraph's advanced control flow.

In [ ]:
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langgraph.graph import StateGraph
from langchain.schema import HumanMessage
from typing import TypedDict, List, Dict

# This is a mock implementation - replace with actual API key if using
# search = GoogleSearchAPIWrapper()
class MockSearch:
    def run(self, query):
        return f"Mock search results for: {query}"

search = MockSearch()

# Use LangChain for tools and integrations
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful for searching the internet"
    )
]

# Define our state using TypedDict
class ResearchState(TypedDict):
    query: str
    search_results: str
    refined_results: str
    final_answer: str

# Define nodes in the graph
def search_for_information(state: ResearchState):
    """Search for information based on the query."""
    query = state["query"]
    search_results = search.run(query)
    state["search_results"] = search_results
    return state

def analyze_search_results(state: ResearchState):
    """Analyze and refine search results."""
    llm = clients["langchain_llm"]  # Use the selected LLM client
    prompt = f"""Analyze these search results and extract the most relevant information:
    Query: {state['query']}
    Search Results: {state['search_results']}
    """
    response = llm.invoke([HumanMessage(content=prompt)])
    state["refined_results"] = response.content
    return state

def formulate_answer(state: ResearchState):
    """Generate a final answer based on the refined results."""
    llm = clients["langchain_llm"]  # Use the selected LLM client
    prompt = f"""Based on the refined information, provide a comprehensive answer to the query:
    Query: {state['query']}
    Refined Information: {state['refined_results']}
    """
    response = llm.invoke([HumanMessage(content=prompt)])
    state["final_answer"] = response.content
    return state

# Create the graph
workflow = StateGraph(ResearchState)

# Add nodes
workflow.add_node("search", search_for_information)
workflow.add_node("analyze", analyze_search_results)
workflow.add_node("answer", formulate_answer)

# Add edges
workflow.add_edge("search", "analyze")
workflow.add_edge("analyze", "answer")

# Set the entry point
workflow.set_entry_point("search")

# Compile the graph
research_graph = workflow.compile()

# Example execution (commented out)
'''
initial_state = {"query": "What are the latest advancements in fusion energy?", "search_results": "", "refined_results": "", "final_answer": ""}
result = research_graph.invoke(initial_state)
print("Final Answer:")
print(result["final_answer"])
'''

## Setting API Keys

For these examples to work, you'll need to set your API keys for the LLM providers you're using.

In [ ]:
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # Replace with your actual API key

## Example 1: LangChain + LangGraph Hybrid

This example combines LangChain's tools and retrievers with LangGraph's workflow control.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langgraph.graph import StateGraph
from typing import TypedDict, List, Dict, Any

# Define our state type
class ResearchState(TypedDict):
    query: str
    needs_research: bool
    research_results: str
    follow_up_questions: List[str]
    answer: str

# Initialize LangChain tools
llm = ChatOpenAI(temperature=0)
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Define LangGraph nodes that use LangChain components

def analyze_query(state: ResearchState) -> ResearchState:
    """Determine if the query needs research."""
    response = llm.invoke([
        HumanMessage(content=f"Does this query require factual research? Answer with just Yes or No: {state['query']}")
    ])
    state["needs_research"] = "yes" in response.content.lower()
    return state

def perform_research(state: ResearchState) -> ResearchState:
    """Use LangChain's Wikipedia tool to gather information."""
    try:
        research_results = wikipedia.run(state["query"])
        state["research_results"] = research_results
    except Exception as e:
        state["research_results"] = f"Error performing research: {str(e)}"
    return state

def generate_questions(state: ResearchState) -> ResearchState:
    """Generate follow-up questions based on research."""
    prompt = f"""
    Based on this research about: {state['query']}
    Research findings: {state['research_results']}
    
    Generate 3 specific follow-up questions that would help explore this topic further.
    Return ONLY the questions as a numbered list.
    """
    response = llm.invoke([HumanMessage(content=prompt)])
    
    # Parse the questions
    questions = response.content.strip().split('\n')
    state["follow_up_questions"] = questions
    return state

def formulate_answer(state: ResearchState) -> ResearchState:
    """Create a comprehensive answer using research results."""
    if state["needs_research"]:
        prompt = f"""
        Query: {state['query']}
        Research findings: {state['research_results']}
        
        Based on this information, provide a comprehensive and accurate answer to the query.
        """
    else:
        prompt = f"Answer this query based on your existing knowledge: {state['query']}"
        
    response = llm.invoke([HumanMessage(content=prompt)])
    state["answer"] = response.content
    return state

# Build the LangGraph workflow
workflow = StateGraph(ResearchState)

# Add nodes
workflow.add_node("analyze", analyze_query)
workflow.add_node("research", perform_research)
workflow.add_node("generate_questions", generate_questions)
workflow.add_node("answer", formulate_answer)

# Add conditional edges
workflow.add_edge("analyze", "research", condition=lambda s: s["needs_research"])
workflow.add_edge("analyze", "answer", condition=lambda s: not s["needs_research"])
workflow.add_edge("research", "generate_questions")
workflow.add_edge("generate_questions", "answer")

# Set entry point
workflow.set_entry_point("analyze")

# Compile graph
graph = workflow.compile()

# Run the hybrid workflow
result = graph.invoke({
    "query": "What were the major contributions of Marie Curie to science?",
    "needs_research": False,
    "research_results": "",
    "follow_up_questions": [],
    "answer": ""
})

print(f"Query: {result['query']}")
print(f"Needed research: {result['needs_research']}")
print("\nFollow-up questions:")
for q in result['follow_up_questions']:
    print(f"- {q}")
print("\nAnswer:")
print(result['answer'])

## Example 2: AutoGen + CrewAI Hybrid

This example combines AutoGen's conversational capabilities with CrewAI's role-based approach.

In [ ]:
import autogen
from crewai import Agent as CrewAgent, Task, Crew
from langchain_openai import ChatOpenAI

# Configure AutoGen
config_list = [
    {
        'model': 'gpt-4',
        'api_key': os.environ.get("OPENAI_API_KEY")
    }
]

# Set up AutoGen conversation agents
autogen_assistant = autogen.AssistantAgent(
    name="Conversational_Assistant",
    llm_config={"config_list": config_list},
    system_message="You help users refine their research questions to be specific and answerable."
)

autogen_user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    human_input_mode="NEVER"  # No human in the loop for this example
)

# Function to use AutoGen for query refinement
def refine_query_with_autogen(query):
    autogen_user_proxy.initiate_chat(
        autogen_assistant,
        message=f"Please help refine this research question to be more specific and answerable: '{query}'"
    )
    # Extract the refined query from the last assistant message
    conversation = autogen_user_proxy.chat_messages[autogen_assistant]
    refined_query = conversation[-1]["content"]
    return refined_query

# Set up CrewAI agents
llm = ChatOpenAI(model="gpt-4")

researcher = CrewAgent(
    role="Research Specialist",
    goal="Find comprehensive and accurate information",
    backstory="You are an expert researcher with a talent for finding relevant information.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

writer = CrewAgent(
    role="Content Writer",
    goal="Create engaging and informative content",
    backstory="You are a skilled writer who transforms complex information into clear, engaging content.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# Function to use CrewAI for research and writing
def research_and_write_with_crewai(query):
    # Define tasks for the crew
    research_task = Task(
        description=f"Research this topic thoroughly: {query}",
        expected_output="Comprehensive research findings with citations",
        agent=researcher
    )
    
    writing_task = Task(
        description="Create an informative article based on the research findings",
        expected_output="Well-structured article with introduction, main points, and conclusion",
        agent=writer,
        context=[research_task]
    )
    
    # Create and run the crew
    crew = Crew(
        agents=[researcher, writer],
        tasks=[research_task, writing_task],
        verbose=1
    )
    
    result = crew.kickoff()
    return result

# Hybrid workflow combining AutoGen and CrewAI
def hybrid_research_process(query):
    print(f"Original query: {query}")
    
    # Step 1: Use AutoGen to refine the query through conversation
    refined_query = refine_query_with_autogen(query)
    print(f"\nRefined query: {refined_query}")
    
    # Step 2: Use CrewAI to research and write content
    result = research_and_write_with_crewai(refined_query)
    print("\nFinal result:")
    print(result)
    
    return result

# Example usage
hybrid_research_process("Tell me about AI agents")

## Example 3: Custom Orchestration Layer

This example demonstrates a custom orchestration layer that selectively uses different frameworks based on the task requirements.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
import autogen
from enum import Enum

# Define task types
class TaskType(Enum):
    RESEARCH = "research"
    CONVERSATION = "conversation"
    CODE_GENERATION = "code_generation"
    CONTENT_CREATION = "content_creation"

class Task:
    def __init__(self, type: TaskType, query: str, context=None):
        self.type = type
        self.query = query
        self.context = context or {}

class CustomOrchestrator:
    def __init__(self):
        # Initialize LLM
        self.llm = ChatOpenAI()
        
        # Set up LangChain tools
        self.wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
        
        # Set up AutoGen agents
        config_list = [
            {
                'model': 'gpt-4',
                'api_key': os.environ.get("OPENAI_API_KEY")
            }
        ]
        
        self.coding_agent = autogen.AssistantAgent(
            name="Coding_Assistant",
            llm_config={"config_list": config_list},
            system_message="You are an expert programmer. Write clean, efficient code."
        )
        
        self.user_proxy = autogen.UserProxyAgent(
            name="User_Proxy",
            human_input_mode="NEVER",
            code_execution_config={"work_dir": "coding_workspace", "use_docker": False}
        )
    
    def process_task(self, task: Task):
        """Process a task using the appropriate framework."""
        if task.type == TaskType.RESEARCH:
            return self._handle_research_task(task)
        elif task.type == TaskType.CONVERSATION:
            return self._handle_conversation_task(task)
        elif task.type == TaskType.CODE_GENERATION:
            return self._handle_code_generation_task(task)
        elif task.type == TaskType.CONTENT_CREATION:
            return self._handle_content_creation_task(task)
        else:
            return {"error": f"Unknown task type: {task.type}"}
    
    def _handle_research_task(self, task: Task):
        """Handle research using LangChain tools."""
        try:
            research_results = self.wikipedia.run(task.query)
            
            # Summarize the research results
            prompt = f"""
            Summarize the following research findings about: {task.query}
            
            {research_results}
            
            Provide a concise but comprehensive summary.
            """
            
            summary_response = self.llm.invoke([HumanMessage(content=prompt)])
            
            return {
                "raw_results": research_results,
                "summary": summary_response.content
            }
        except Exception as e:
            return {"error": f"Research error: {str(e)}"}
    
    def _handle_conversation_task(self, task: Task):
        """Handle conversation using direct LLM calls."""
        try:
            response = self.llm.invoke([HumanMessage(content=task.query)])
            return {"response": response.content}
        except Exception as e:
            return {"error": f"Conversation error: {str(e)}"}
    
    def _handle_code_generation_task(self, task: Task):
        """Handle code generation using AutoGen."""
        try:
            self.user_proxy.initiate_chat(
                self.coding_agent,
                message=task.query
            )
            
            # Extract the generated code from the conversation
            conversation = self.user_proxy.chat_messages[self.coding_agent]
            code_response = conversation[-1]["content"]
            
            return {"generated_code": code_response}
        except Exception as e:
            return {"error": f"Code generation error: {str(e)}"}
    
    def _handle_content_creation_task(self, task: Task):
        """Handle content creation using a tailored prompt."""
        try:
            prompt = f"""
            Create content on the following topic: {task.query}
            
            Style: {task.context.get('style', 'informative')}
            Length: {task.context.get('length', 'medium')}
            Target audience: {task.context.get('audience', 'general')}
            
            Create well-structured content that is engaging and valuable for the target audience.
            """
            
            response = self.llm.invoke([HumanMessage(content=prompt)])
            return {"content": response.content}
        except Exception as e:
            return {"error": f"Content creation error: {str(e)}"}

# Create orchestrator
orchestrator = CustomOrchestrator()

# Example usage for different task types

# Research task
research_task = Task(
    type=TaskType.RESEARCH,
    query="What are the environmental impacts of quantum computing?"
)
research_result = orchestrator.process_task(research_task)
print("\n=== RESEARCH TASK RESULT ===")
print(research_result.get('summary', research_result.get('error')))

# Code generation task
code_task = Task(
    type=TaskType.CODE_GENERATION,
    query="Write a Python function to find the prime numbers in a given range using the Sieve of Eratosthenes algorithm."
)
code_result = orchestrator.process_task(code_task)
print("\n=== CODE GENERATION TASK RESULT ===")
print(code_result.get('generated_code', code_result.get('error')))

# Content creation task
content_task = Task(
    type=TaskType.CONTENT_CREATION,
    query="The future of renewable energy",
    context={"style": "authoritative", "length": "short", "audience": "policymakers"}
)
content_result = orchestrator.process_task(content_task)
print("\n=== CONTENT CREATION TASK RESULT ===")
print(content_result.get('content', content_result.get('error')))

## Example 4: Custom Agent with Performance Optimizations

This example demonstrates a custom agent implementation with various performance optimizations.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
import time
import hashlib
import json

class OptimizedAgent:
    def __init__(self, name, model="gpt-4", cache_enabled=True):
        self.name = name
        self.llm = ChatOpenAI(model=model)
        self.cache = {}
        self.cache_enabled = cache_enabled
        self.request_count = 0
        self.token_count = 0
        
    def _get_cache_key(self, prompt):
        """Generate a cache key for a prompt."""
        return hashlib.md5(prompt.encode()).hexdigest()
    
    def _optimize_prompt(self, prompt, max_tokens=4000):
        """Optimize a prompt to reduce token usage."""
        if len(prompt) <= max_tokens:
            return prompt
        
        # Simple truncation strategy
        return prompt[:max_tokens] + "... [truncated]"
    
    def generate(self, prompt, use_cache=True):
        """Generate a response with caching and performance tracking."""
        start_time = time.time()
        
        # Optimize the prompt
        optimized_prompt = self._optimize_prompt(prompt)
        
        # Check cache if enabled
        if self.cache_enabled and use_cache:
            cache_key = self._get_cache_key(optimized_prompt)
            if cache_key in self.cache:
                print(f"[{self.name}] Cache hit!")
                return self.cache[cache_key], 0, 0
        
        # Make the API call
        try:
            self.request_count += 1
            response = self.llm.invoke([HumanMessage(content=optimized_prompt)])
            content = response.content
            
            # Estimate token count (very rough estimation)
            estimated_tokens = len(optimized_prompt.split()) + len(content.split())
            self.token_count += estimated_tokens
            
            # Cache the result if enabled
            if self.cache_enabled and use_cache:
                cache_key = self._get_cache_key(optimized_prompt)
                self.cache[cache_key] = content
            
            elapsed_time = time.time() - start_time
            return content, estimated_tokens, elapsed_time
            
        except Exception as e:
            print(f"Error: {str(e)}")
            return f"Error: {str(e)}", 0, time.time() - start_time
    
    def get_stats(self):
        """Get performance statistics."""
        return {
            "name": self.name,
            "requests": self.request_count,
            "estimated_tokens": self.token_count,
            "cache_size": len(self.cache),
            "cache_enabled": self.cache_enabled
        }
    
    def clear_cache(self):
        """Clear the response cache."""
        cache_size = len(self.cache)
        self.cache = {}
        return f"Cleared {cache_size} items from cache."

# Create an optimized agent
agent = OptimizedAgent("OptimizedAssistant")

# Example 1: First request (no cache)
prompt1 = "Explain the concept of quantum computing in simple terms."
response1, tokens1, time1 = agent.generate(prompt1)

print(f"Response 1 (took {time1:.2f}s, ~{tokens1} tokens):")
print(response1[:300] + "..." if len(response1) > 300 else response1)
print("\n" + "-"*50 + "\n")

# Example 2: Repeated request (should use cache)
response2, tokens2, time2 = agent.generate(prompt1)

print(f"Response 2 (took {time2:.2f}s, ~{tokens2} tokens):")
print(f"Cache performance improvement: {time1/time2:.2f}x faster")
print("\n" + "-"*50 + "\n")

# Example 3: New request with long prompt (will be optimized)
long_prompt = "Explain the history of artificial intelligence" + " very thoroughly and in detail" * 100
response3, tokens3, time3 = agent.generate(long_prompt)

print(f"Response 3 (took {time3:.2f}s, ~{tokens3} tokens):")
print(response3[:300] + "..." if len(response3) > 300 else response3)
print("\n" + "-"*50 + "\n")

# Print stats
print("Agent Statistics:")
print(json.dumps(agent.get_stats(), indent=2))

## Example 5: Multi-Framework Pipeline for Document Analysis

This example demonstrates a pipeline that uses multiple frameworks for different stages of document analysis.

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.schema import HumanMessage, Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langgraph.graph import StateGraph
from typing import TypedDict, List, Dict, Any
import autogen

# Sample document
sample_document = """
# Artificial Intelligence: A Modern Approach

## Introduction
Artificial Intelligence (AI) is a field of computer science focused on creating systems capable of performing tasks that typically require human intelligence. These tasks include learning, reasoning, problem-solving, perception, and language understanding.

## History of AI
The field of AI research was founded at a workshop held at Dartmouth College in 1956. The attendees, including John McCarthy, Marvin Minsky, Allen Newell, and Herbert Simon, became the founders and leaders of AI research. They and their students produced programs that were described by the press as "astonishing": computers were solving word problems in algebra, proving logical theorems, and speaking English.

By the middle of the 1960s, research in the U.S. was heavily funded by the Department of Defense, and AI laboratories had been established around the world. AI had achieved success in limited domains, but had difficulty with more general problems.

## Machine Learning
Machine learning is a subset of AI that provides systems the ability to automatically learn and improve from experience without being explicitly programmed. It focuses on the development of computer programs that can access data and use it to learn for themselves.

### Types of Machine Learning
1. **Supervised Learning**: The algorithm is trained on labeled data.
2. **Unsupervised Learning**: The algorithm finds patterns in unlabeled data.
3. **Reinforcement Learning**: The algorithm learns through trial and error, guided by rewards.

## Natural Language Processing
Natural Language Processing (NLP) is a field of AI that deals with the interaction between computers and humans using natural language. The ultimate objective of NLP is to read, understand, and generate human languages in a valuable way.

## Computer Vision
Computer vision is an interdisciplinary field that deals with how computers can be made to gain high-level understanding from digital images or videos. From the perspective of engineering, it seeks to automate tasks that the human visual system can do.

## Ethical Considerations
As AI systems become more powerful, ethical considerations become increasingly important. Key concerns include:

- Privacy and surveillance
- Bias and fairness
- Accountability and transparency
- Economic impact and job displacement

## Future Directions
Future research in AI will likely focus on creating more general systems that can perform a wide variety of tasks, rather than specialized systems for specific applications. Artificial General Intelligence (AGI) remains a long-term goal of many researchers.
"""

class DocumentState(TypedDict):
    document: str
    chunks: List[str]
    topics: List[str]
    summaries: Dict[str, str]
    final_analysis: str

# Define the document analysis pipeline
class DocumentAnalysisPipeline:
    def __init__(self):
        self.llm = ChatOpenAI()
        self.embeddings = OpenAIEmbeddings()
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        
        # Set up AutoGen agents
        config_list = [
            {
                'model': 'gpt-4',
                'api_key': os.environ.get("OPENAI_API_KEY")
            }
        ]
        
        self.analyst_agent = autogen.AssistantAgent(
            name="Document_Analyst",
            llm_config={"config_list": config_list},
            system_message="You are an expert at analyzing documents and extracting key insights."
        )
        
        self.user_proxy = autogen.UserProxyAgent(
            name="User_Proxy",
            human_input_mode="NEVER"
        )
        
        # Build LangGraph workflow
        self.workflow = self._build_workflow()
    
    def _build_workflow(self):
        workflow = StateGraph(DocumentState)
        
        # Add nodes
        workflow.add_node("chunk_document", self.chunk_document)
        workflow.add_node("extract_topics", self.extract_topics)
        workflow.add_node("generate_summaries", self.generate_summaries)
        workflow.add_node("analyze_document", self.analyze_document)
        
        # Add edges
        workflow.add_edge("chunk_document", "extract_topics")
        workflow.add_edge("extract_topics", "generate_summaries")
        workflow.add_edge("generate_summaries", "analyze_document")
        
        # Set entry point
        workflow.set_entry_point("chunk_document")
        
        return workflow.compile()
    
    # LangChain-based document chunking
    def chunk_document(self, state: DocumentState) -> DocumentState:
        chunks = self.text_splitter.split_text(state["document"])
        state["chunks"] = chunks
        return state
    
    # LLM-based topic extraction
    def extract_topics(self, state: DocumentState) -> DocumentState:
        prompt = f"""
        Analyze this document and identify the main topics covered:
        
        {state['document']}
        
        Return ONLY a list of 3-7 main topics as bullet points.
        """
        
        response = self.llm.invoke([HumanMessage(content=prompt)])
        topics = [line.strip().replace('- ', '').replace('* ', '') 
                 for line in response.content.strip().split('\n') 
                 if line.strip() and (line.strip().startswith('- ') or line.strip().startswith('* '))]        
        state["topics"] = topics
        return state
    
    # Generate topic-specific summaries using AutoGen
    def generate_summaries(self, state: DocumentState) -> DocumentState:
        summaries = {}
        
        for topic in state["topics"]:
            # Use AutoGen for detailed analysis of each topic
            self.user_proxy.initiate_chat(
                self.analyst_agent,
                message=f"Analyze this document regarding the topic '{topic}':\n\n{state['document']}\n\nProvide a concise summary of how this topic is covered."
            )
            
            conversation = self.user_proxy.chat_messages[self.analyst_agent]
            topic_summary = conversation[-1]["content"]
            summaries[topic] = topic_summary
        
        state["summaries"] = summaries
        return state
    
    # LangGraph-based final analysis
    def analyze_document(self, state: DocumentState) -> DocumentState:
        topics_text = "\n".join([f"- {topic}" for topic in state["topics"]])
        summaries_text = "\n\n".join([f"## {topic}\n{summary}" for topic, summary in state["summaries"].items()])
        
        prompt = f"""
        Based on the analysis of the document, create a comprehensive overview.
        
        Main topics identified:
        {topics_text}
        
        Topic summaries:
        {summaries_text}
        
        Please provide a final analysis that includes:
        1. An overview of the document's key themes
        2. The relationships between different topics
        3. The most important insights from the document
        4. Any notable gaps or areas for further exploration
        """
        
        response = self.llm.invoke([HumanMessage(content=prompt)])
        state["final_analysis"] = response.content
        return state
    
    def analyze(self, document: str) -> Dict[str, Any]:
        result = self.workflow.invoke({
            "document": document,
            "chunks": [],
            "topics": [],
            "summaries": {},
            "final_analysis": ""
        })
        return result

# Run the document analysis pipeline
pipeline = DocumentAnalysisPipeline()
analysis_result = pipeline.analyze(sample_document)

print("\n=== DOCUMENT ANALYSIS RESULTS ===")
print("\nIdentified Topics:")
for topic in analysis_result["topics"]:
    print(f"- {topic}")

print("\nTopic Summaries (excerpt from first topic):")
first_topic = analysis_result["topics"][0]
first_summary = analysis_result["summaries"][first_topic]
print(f"## {first_topic}")
print(first_summary[:200] + "..." if len(first_summary) > 200 else first_summary)
print("...and summaries for other topics...")

print("\nFinal Analysis:")
print(analysis_result["final_analysis"][:500] + "..." if len(analysis_result["final_analysis"]) > 500 else analysis_result["final_analysis"])

## Conclusion and Best Practices

These examples demonstrate various approaches to building hybrid AI agent systems that combine multiple frameworks. When building your own hybrid systems, consider these best practices:

1. **Choose frameworks based on strengths**: Select each framework for what it does best
2. **Design clean interfaces**: Create clear boundaries between framework-specific components
3. **Implement proper error handling**: Account for failures across framework boundaries
4. **Monitor performance**: Track resource usage and response times for optimization
5. **Build incrementally**: Start simple and add complexity as needed
6. **Document integration points**: Make clear how data flows between framework components

As the field of AI agents continues to evolve, hybrid approaches that combine the strengths of different frameworks will likely become increasingly common in production systems.

## Pattern 2: CrewAI + AutoGen

This example demonstrates how to combine CrewAI's role-based agents with AutoGen's conversational capabilities.

In [ ]:
from crewai import Agent as CrewAgent, Task, Crew
import autogen
from autogen import AssistantAgent, UserProxyAgent

# Create CrewAI agents using our LLM client
researcher = CrewAgent(
    role="Research Analyst",
    goal="Find comprehensive information on given topics",
    backstory="You are an expert researcher with 15 years of experience in data analysis.",
    llm=clients["langchain_llm"]  # Using the selected LLM
)

writer = CrewAgent(
    role="Content Writer",
    goal="Transform research into engaging content",
    backstory="You are a skilled writer who can explain complex topics clearly.",
    llm=clients["langchain_llm"]  # Using the selected LLM
)

# Create AutoGen agents using our AutoGen config
assistant = AssistantAgent(
    name="AI_Assistant",
    llm_config=clients["autogen_config"],
    system_message="You help users refine their research questions and provide guidance."
)

user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER"  # Don't ask for human input in this example
)

# Function to connect CrewAI and AutoGen
def hybrid_research_process(topic):
    """Combine CrewAI and AutoGen to research a topic and create content."""
    # Step 1: Use AutoGen to refine the research question
    question_refinement = user_proxy.initiate_chat(
        assistant,
        message=f"Help me refine this research topic: '{topic}'. Please suggest a more specific research question."
    )
    
    # Extract the refined question (in a real implementation, you would parse the chat history)
    refined_question = f"Refined version of: {topic}"  # Placeholder
    
    # Step 2: Use CrewAI for research and content creation
    research_task = Task(
        description=f"Research this topic thoroughly: {refined_question}. Find key information, statistics, and insights.",
        agent=researcher
    )
    
    writing_task = Task(
        description="Using the research findings, create an engaging article that explains the topic clearly.",
        agent=writer,
        context=[research_task]  # This task depends on the research task
    )
    
    # Form a crew with these agents and tasks
    crew = Crew(
        agents=[researcher, writer],
        tasks=[research_task, writing_task]
    )
    
    # Execute the workflow (commented out)
    # result = crew.kickoff()
    # return result
    
    # For demonstration, return a placeholder
    return f"Research and content creation process completed for: {refined_question}"

# Example usage (commented out)
'''
result = hybrid_research_process("The impact of artificial intelligence on healthcare")
print(result)
'''

## Pattern 3: Custom Orchestration Layer

This example demonstrates how to create a custom orchestration layer that integrates multiple frameworks.

In [ ]:
class CustomOrchestrator:
    """A custom orchestrator that integrates multiple agent frameworks."""
    
    def __init__(self, provider="openai"):
        # Set up clients based on provider choice
        self.clients = create_clients(provider)
        
        # Set up components from different frameworks
        self._setup_langchain_components()
        self._setup_autogen_components()
        self._setup_crewai_components()
        self._setup_langgraph_workflow()
        
    def _setup_langchain_components(self):
        """Set up LangChain tools and components."""
        from langchain.agents import Tool
        from langchain.utilities import GoogleSearchAPIWrapper
        
        # Mock search for demonstration
        self.search = MockSearch()
        self.langchain_tools = [
            Tool(
                name="Search",
                func=self.search.run,
                description="Useful for searching the internet"
            )
        ]
    
    def _setup_autogen_components(self):
        """Set up AutoGen agents."""
        self.autogen_assistant = AssistantAgent(
            name="AI_Assistant",
            llm_config=self.clients["autogen_config"],
            system_message="You help refine questions and provide guidance."
        )
        
        self.autogen_user = UserProxyAgent(
            name="User",
            human_input_mode="NEVER"  # Don't ask for human input in this example
        )
    
    def _setup_crewai_components(self):
        """Set up CrewAI agents and crew."""
        self.researcher = CrewAgent(
            role="Research Analyst",
            goal="Find comprehensive information",
            backstory="You are an expert researcher.",
            llm=self.clients["langchain_llm"]
        )
        
        self.writer = CrewAgent(
            role="Content Writer",
            goal="Create engaging content",
            backstory="You excel at creating readable content.",
            llm=self.clients["langchain_llm"]
        )
        
        # Tasks will be created dynamically based on the query
        
    def _setup_langgraph_workflow(self):
        """Set up LangGraph workflow."""
        # Define a simple workflow that will be used for processing
        # This is a placeholder - in a real implementation, you would define a more complex graph
        workflow = StateGraph(ResearchState)
        workflow.add_node("search", search_for_information)
        workflow.add_node("analyze", analyze_search_results)
        workflow.compile()
        self.workflow = workflow
    
    def process_task(self, task_type, query):
        """Process a task using the appropriate framework."""
        if task_type == "research":
            return self._handle_research_task(query)
        elif task_type == "content_creation":
            return self._handle_content_task(query)
        elif task_type == "conversation":
            return self._handle_conversation_task(query)
        else:
            return f"Unknown task type: {task_type}"
    
    def _handle_research_task(self, query):
        """Handle research tasks using LangGraph + LangChain."""
        initial_state = {"query": query, "search_results": "", "refined_results": "", "final_answer": ""}
        # In a real implementation, you would run the graph
        # result = self.workflow.invoke(initial_state)
        # return result["final_answer"]
        return f"Research results for: {query} (using LangGraph + LangChain)"
    
    def _handle_content_task(self, query):
        """Handle content creation tasks using CrewAI."""
        research_task = Task(
            description=f"Research this topic: {query}",
            agent=self.researcher
        )
        
        writing_task = Task(
            description=f"Create content about: {query}",
            agent=self.writer,
            context=[research_task]
        )
        
        crew = Crew(
            agents=[self.researcher, self.writer],
            tasks=[research_task, writing_task]
        )
        
        # In a real implementation, you would run the crew
        # result = crew.kickoff()
        # return result
        return f"Content created for: {query} (using CrewAI)"
    
    def _handle_conversation_task(self, query):
        """Handle conversation tasks using AutoGen."""
        # In a real implementation, you would initiate a conversation
        # self.autogen_user.initiate_chat(
        #     self.autogen_assistant,
        #     message=query
        # )
        return f"Conversation about: {query} (using AutoGen)"

# Example usage (commented out)
'''
# Create the orchestrator
orchestrator = CustomOrchestrator(provider="openai")  # Change to "azure" to use Azure OpenAI

# Process different types of tasks
research_result = orchestrator.process_task("research", "What are the emerging trends in quantum computing?")
print(research_result)

content_result = orchestrator.process_task("content_creation", "The future of renewable energy")
print(content_result)

conversation_result = orchestrator.process_task("conversation", "Explain the concept of artificial general intelligence")
print(conversation_result)
'''

## Conclusion

In this notebook, we've explored how to build hybrid AI agent systems that combine multiple frameworks:

1. **LangChain + LangGraph**: Leveraging LangChain's rich ecosystem of tools with LangGraph's sophisticated control flow

2. **CrewAI + AutoGen**: Combining CrewAI's intuitive role definitions with AutoGen's powerful conversation mechanisms

3. **Custom Orchestration**: Building a custom orchestration layer that selectively uses different frameworks based on the task

These hybrid approaches allow you to leverage the strengths of different frameworks while avoiding their individual limitations, creating more powerful and flexible AI agent systems.